In [8]:
#PASSO 1: Tokenização
#Aqui a gente transforma o texto em números

import numpy as np

#Aqui abaixo na variável texto precisa fazer a injeção de textos das fontes que quiser.
texto = "DOCUMENTO MOCK\nCapítulo 1: Introdução  \nEste é um texto fictício com acentos (á, é, ã, ç) e símbolos (@#%) \nUse para testar seu tokenizador e embeddings!\nAutor: Você."
texto = texto.split(' ')
vocabulario = sorted(list(set(texto)))
print(vocabulario)

['', '\nEste', '\nUse', '(@#%)', '(á,', '1:', 'DOCUMENTO', 'Introdução', 'MOCK\nCapítulo', 'Você.', 'acentos', 'com', 'e', 'embeddings!\nAutor:', 'fictício', 'para', 'seu', 'símbolos', 'testar', 'texto', 'tokenizador', 'um', 'ã,', 'ç)', 'é', 'é,']


In [9]:
#Seta cada palavra a um token e vice versa
token_para_id = {c: i for i, c in enumerate (vocabulario)}
id_para_token = {i: c for i, c in enumerate (vocabulario)}
print(token_para_id)
print(id_para_token)


{'': 0, '\nEste': 1, '\nUse': 2, '(@#%)': 3, '(á,': 4, '1:': 5, 'DOCUMENTO': 6, 'Introdução': 7, 'MOCK\nCapítulo': 8, 'Você.': 9, 'acentos': 10, 'com': 11, 'e': 12, 'embeddings!\nAutor:': 13, 'fictício': 14, 'para': 15, 'seu': 16, 'símbolos': 17, 'testar': 18, 'texto': 19, 'tokenizador': 20, 'um': 21, 'ã,': 22, 'ç)': 23, 'é': 24, 'é,': 25}
{0: '', 1: '\nEste', 2: '\nUse', 3: '(@#%)', 4: '(á,', 5: '1:', 6: 'DOCUMENTO', 7: 'Introdução', 8: 'MOCK\nCapítulo', 9: 'Você.', 10: 'acentos', 11: 'com', 12: 'e', 13: 'embeddings!\nAutor:', 14: 'fictício', 15: 'para', 16: 'seu', 17: 'símbolos', 18: 'testar', 19: 'texto', 20: 'tokenizador', 21: 'um', 22: 'ã,', 23: 'ç)', 24: 'é', 25: 'é,'}


In [10]:
#Tokenização
tokens = [token_para_id[c] for c in texto] #Reescreve o texto inicial como tokens
print("Tokens:", tokens)

Tokens: [6, 8, 5, 7, 0, 1, 24, 21, 19, 14, 11, 10, 4, 25, 22, 23, 12, 17, 3, 2, 15, 18, 16, 20, 12, 13, 9]


In [11]:
#PASSO 2: Embedding (Vetores de Palavras)
#Cada token vira um vetor de dimensão "d_model"

d_model = 256 #Isso é o tanto de números que vão representar uma letra! (Sim, precisa ser um número razoavelmente alto porque precisa diferenciar acentuações, maiúsculas, minúsculas e todo tipo de variação que existir de um caractere para outro)
vocab_size = len(vocabulario)

#Embeddings aleatórios (normalmente aprendidos durante o treino)
embedding = np.random.randn(vocab_size, d_model)

#Transforma tokens em vetores
token_embeddings = embedding[tokens] #Shape: [len(tokens), d_model]
print(token_embeddings)

[[-0.48866214  0.98030233  1.44460472 ... -0.92668625  1.03506295
   1.1978157 ]
 [-0.29382744 -0.32905208  0.31011814 ...  0.58765784  0.93255109
  -0.62690546]
 [-0.61953972 -0.23887039  0.43482099 ... -0.23239204 -1.05397606
  -0.76669491]
 ...
 [-0.87887742 -1.83396644 -1.27937437 ... -1.52764746  0.65789263
   0.68749182]
 [ 1.13119758 -0.46751032  1.42776106 ... -1.28724562  1.08805122
  -0.37854147]
 [-0.4021161  -0.23834376 -0.12063326 ...  0.32063256 -0.51398253
  -0.3826067 ]]


In [12]:
#PASSO 3: Mecanismo de Atenção (Self-Attention)
#Calcula a relação entre tokens, versão simplificada:
def attention(Q, K, V):
    # Q, K, V são matrizes de consulta, chave e valor
    d_k = Q.shape[-1]
    scores = np.dot(Q, K.T) / np.sqrt(d_k)  # Similaridade: Calcula o "quão bem" a palavra Q se relaciona com as outras (K)
    weights = np.exp(scores) / np.sum(np.exp(scores), axis=-1, keepdims=True)  # Softmax: Transforma os scores em probabilidades (soma = 1)
    return np.dot(weights, V)  # Combinação: Combina os embeddings das palavras (V) usando os pesos de atenção

# Exemplo:
Q = token_embeddings  # Consulta: O que cada palavra está buscando
K = token_embeddings  # Chave: Como cada palavra se descreve
V = token_embeddings  # Valor: Informação real de cada palavra
saida_attention = attention(Q, K, V)


In [13]:
#PASSO 4: Rede Neural Feed-Forward
#Uma MLP simples para processar os vetores após o mecanismo de atenção
def feed_forward(x, W1, b1, W2, b2):
    hidden = np.maximum(0, np.dot(x, W1) + b1)  # Camada oculta com ReLU
    return np.dot(hidden, W2) + b2 # Camada de saída

# Pesos aleatórios (em um modelo real, seriam aprendidos)
W1 = np.random.randn(d_model, 4 * d_model) # Pesos da camada oculta
b1 = np.random.randn(4 * d_model) # Vieses da camada oculta
W2 = np.random.randn(4 * d_model, d_model) # Pesos da camada de saída
b2 = np.random.randn(d_model) # Vieses da camada de saída

saida_ff = feed_forward(saida_attention, W1, b1, W2, b2)

print(saida_attention)

[[-0.48865919  0.98029643  1.44459362 ... -0.92668003  1.03505686
   1.19781057]
 [-0.29376997 -0.32894462  0.31008805 ...  0.58761112  0.93246826
  -0.62687289]
 [-0.61953837 -0.23886899  0.43482038 ... -0.23239149 -1.05397395
  -0.76669356]
 ...
 [-0.87887662 -1.83396444 -1.27937319 ... -1.5276455   0.65789216
   0.68749164]
 [ 1.13119651 -0.46750938  1.42775959 ... -1.28724406  1.08805013
  -0.37854058]
 [-0.40211205 -0.23833944 -0.12063269 ...  0.32063204 -0.51397904
  -0.38260605]]


In [14]:
#PASSO 5: Treinamento
#Para treinar precisaríamos
#1 - Dados: Um corpus de texto (ex.: livros do Projeto Gutenberg).
#2 - Função de perda (Loss Function): Cross-entropy entre previsões e tokens reais.
#3 - Backpropagation: Implementar gradientes manualmente (NumPy não tem autograd).

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True)) # Evita overflow
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

def cross_entropy(y_pred, y_true):
    m = y_true.shape[0] #Número de exemplos
    log_probs = -np.log(y_pred[range(m), y_true]) # Penaliza previsões erradas
    return np.sum(log_probs) / m # Média do erro

# Exemplo fictício:
y_pred = softmax(np.random.randn(len(tokens), vocab_size))  # Previsões aleatórias
y_true = np.array(tokens)  # Tokens reais
loss = cross_entropy(y_pred, y_true)
print("Loss:", loss) # Quanto menor, melhor

Loss: 3.854085529317129


In [23]:
import numpy as np

# === PASSO 1: Tokenização ===
texto = "O rato roeu a roupa do rei de roma."
palavras = texto.split()  # Divide o texto em palavras
vocabulario = sorted(list(set(palavras)))  # Vocabulário único

# Mapeamento palavra → ID e vice-versa
token_para_id = {palavra: i for i, palavra in enumerate(vocabulario)}
id_para_token = {i: palavra for i, palavra in enumerate(vocabulario)}

# Tokeniza o texto
tokens = [token_para_id[palavra] for palavra in palavras]

# === PASSO 2: Embeddings ===
d_model = 64  # Dimensão dos vetores
vocab_size = len(vocabulario)

# Cria a embedding_matrix (aleatória inicialmente)
embedding_matrix = np.random.randn(vocab_size, d_model)  # <--- AGORA DEFINIDA!

# Obtém os embeddings das palavras do texto
token_embeddings = embedding_matrix[tokens]  # Shape: [n_palavras, d_model]

# === PASSO 3: Self-Attention ===
def attention(Q, K, V):
    d_k = Q.shape[-1]
    scores = np.dot(Q, K.T) / np.sqrt(d_k)
    weights = np.exp(scores) / np.sum(np.exp(scores), axis=-1, keepdims=True)
    return np.dot(weights, V)

saida_attention = attention(token_embeddings, token_embeddings, token_embeddings)

# === PASSO 4: Feed-Forward ===
def feed_forward(x, W1, b1, W2, b2):
    hidden = np.maximum(0, np.dot(x, W1) + b1)  # ReLU
    return np.dot(hidden, W2) + b2

# Inicializa pesos
W1 = np.random.randn(d_model, 4 * d_model)
b1 = np.random.randn(4 * d_model)
W2 = np.random.randn(4 * d_model, d_model)
b2 = np.random.randn(d_model)

saida_ff = feed_forward(saida_attention, W1, b1, W2, b2)

# === PASSO 5: Gerar Resposta ===
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

# Pega o último vetor (representação da frase toda)
vetor_frase = saida_ff[-1]

# Calcula probabilidades para todas as palavras do vocabulário
logits = np.dot(vetor_frase, embedding_matrix.T)
probs = softmax(logits)

# Escolhe a palavra mais provável como resposta
resposta_id = np.argmax(probs)
resposta = id_para_token[resposta_id]

print(f"Texto original: {texto}")
print(f"Resposta gerada: {resposta}")

Texto original: O rato roeu a roupa do rei de roma.
Resposta gerada: rei
